In [1]:
!pip install --upgrade pip
!3
!pip install stability-sdk
!pip install python_pptx
!pip install transformers
#!pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import openai
import json
import os
import io
import requests
import shutil
from transformers import pipeline
from pptx import Presentation
from pptx.util import Inches
from PIL import Image
from pptx.util import Pt
from stability_sdk import client
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
from pptx.enum.shapes import MSO_SHAPE
from pptx.enum.text import MSO_AUTO_SIZE
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

os.environ['STABILITY_HOST'] = "grpc.stability.ai:443"
os.environ['STABILITY_KEY'] = "YOUR_API_KEY"
api_key_unsplash = "YOUR_API_KEY"
api_key_pixabay = "YOUR_API_KEY"
api_key_pexels = "YOUR_API_KEY"
openai.api_key="YOUR_API_KEY"
api_key_flickr = "YOUR_API_KEY"
nli_pipeline = pipeline("text-classification", model="roberta-large-mnli")

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [3]:

industry = "Digital Marketing" 
topic = "Advantages and disadvantages of gorilla marketing"
purpose = "To educate students"
tone = "professional and informative"


In [4]:
def generate_ppt(industry:str, purpose: str, topic: str, tone: str):
  prompt = """Create content for a presentation for following industry, purpose, on the following topic in the given tone. Also include a title slide, give outline of the slide, description of each slide, a pictorial representation describing the image with details to be included in each slide, along with bullet points and include a conclusion slide. Give the output in the format of a python dictionary, and use " instead of ' wherever required.
""
Example:
Industry: Dental Hygiene
Purpose: To pitch to kids
Topic: Toothbrush
Tone: Professional
{"title_slide": {
        "title": "Discover the Magic of Toothbrushes",
        "subtitle": "A fun journey to healthy teeth"
    },
    "outline_slide": {
        "title": "Presentation Outline",
        "bullet_points": [
            "Why do we need to brush our teeth?",
            "Introducing the toothbrush",
            "Types of toothbrushes",
            "How to choose the right toothbrush",
            "Proper brushing techniques",
            "Conclusion"
        ]
    },
    "slide1": {
        "title": "Why do we need to brush our teeth?",
        "description": "Brushing is a very important habit to keep.",
        "image": "A cartoon image of healthy teeth smiling",
        "bullet_points": [
            "Remove food particles",
            "Prevent tooth decay",
            "Keep gums healthy",
            "Avoid bad breath",
            "Maintain overall oral hygiene"
        ]
    },
    "conclusion_slide": {
        "title": "Conclusion",
        "description": "Brushing keeps your mouth clean and free from diseases.",
        "image": "A happy child with a toothbrush and sparkling clean teeth",
        "bullet_points": [
            "Choose the right toothbrush",
            "Learn proper brushing techniques",
            "Keep your teeth and gums healthy",
            "Enjoy a beautiful smile"
        ]
    }
}
""
"""
  response = openai.ChatCompletion.create(
  model="gpt-4",
  messages=[
      {"role": "user", "content": f"{prompt}\nIndutry:{industry}\nPurpose:{purpose}\nTopic{topic}\nTone:{tone}\n"}
      ],
  temperature = 0.8,
  n = 1,
  max_tokens = 1256
  )

  return json.loads(response["choices"][0]["message"]["content"])

In [5]:
def get_ai_img_for_ppt(prompt: str):
  stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'], 
    verbose=False, 
    engine="stable-diffusion-768-v2-1")
  
  prompt_str = [generation.Prompt(text=f"epic award winning photo of {prompt}",parameters=generation.PromptParameters(weight=1)), generation.Prompt(text="cinematic atmosphere, high contrast",parameters=generation.PromptParameters(weight=1.9)), generation.Prompt(text="best quality, photorealistic, cinematic, movie quality, octane render",parameters=generation.PromptParameters(weight=1.2)), generation.Prompt(text="dark theme, vfx post production, rtx ray tracing lighting, ultra realistc, 8k detailed post processing, unreal engine, hyper-detailed, cinematic 8k, canon eos 5D",parameters=generation.PromptParameters(weight=1.3)), generation.Prompt(text="cgi, fake, render, painting, illustration, unrealistic, not realistic, cropped, lowres, low resolution, bad quality",parameters=generation.PromptParameters(weight=-1.8)), generation.Prompt(text="3D render, worst quality, low quality, bad lighting, average lighting",parameters=generation.PromptParameters(weight=-1.7))] 
  
  answers = stability_api.generate(
    prompt=prompt_str,
    seed=992446758, 
    steps=120, 
    cfg_scale=14.0, 
    width=512, 
    height=512, 
    samples=1, 
    sampler=generation.SAMPLER_K_DPMPP_2M)
  
  for resp in answers:
    for artifact in resp.artifacts:
        if artifact.type == generation.ARTIFACT_IMAGE:
            return io.BytesIO(artifact.binary)

In [6]:
def get_unsplash_img_for_ppt(query: str): 
  image_url = ""

  response = requests.get(f'https://api.unsplash.com/search/photos?query={query}&client_id={api_key_unsplash}')

  if response.status_code == 200:
      data = response.json()
      image_url = data['results'][0]['urls']['regular']

  return image_url

In [7]:
def get_pixabay_img_for_ppt(query: str): 
  image_url = ""

  response = requests.get(f"https://pixabay.com/api/?key={api_key_pixabay}&q={query}")

  if response.status_code == 200:
      data = response.json()
      image_url = [image['webformatURL'] for image in data['hits']]

  return image_url[0]

In [8]:
def get_pexels_img_for_ppt(query: str):
  image_url = ""
  headers = {
    "Authorization": api_key_pexels
  }
  response = requests.get(f"https://api.pexels.com/v1/search?query={query}", headers=headers)

  if response.status_code == 200:
    data = response.json()
    image_url = [photo['src']['medium'] for photo in data['photos']]

  return image_url[0]

In [9]:
def get_flickr_img_for_ppt(query: str):
  per_page = 1
  format_type = 'json'
  nojsoncallback = 1

  response = requests.get(f"https://api.flickr.com/services/rest/?method=flickr.photos.search&api_key={api_key_flickr}&text={query}&per_page={per_page}&format={format_type}&nojsoncallback={nojsoncallback}")

  data = json.loads(response.text)
  if data['stat'] == 'ok':
    photo = data['photos']['photo'][0]
    farm_id = photo['farm']
    server_id = photo['server']
    photo_id = photo['id']
    secret = photo['secret']
    size = 'm'  

    image_url = f"https://farm{farm_id}.staticflickr.com/{server_id}/{photo_id}_{secret}_{size}.jpg"

  return image_url

In [10]:
def is_question(sentence):
    hypothesis = "This must be treated as a question."

    result = nli_pipeline(sentence, hypothesis, top_k=1)
    if result[0]["score"] > 0.55:
        return True
    else:
        return False

In [11]:
def get_question_explainer(prompt: str, topic: str):
  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "user", "content": f"Give factual answer on {prompt} from perspective of {topic} in 1 sentence. No incomplete sentences."}
      ],
      temperature = 0.8,
      n=1,
      max_tokens=60,
      frequency_penalty=1.19,
      presence_penalty=1.68
  )
  return response["choices"][0]["message"]["content"]

In [12]:
def get_point_explainer(prompt: str):
  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
          {"role": "user", "content": f"Give a 1 sentence explanation on the below topic. No incomplete sentences.\nTopic:{prompt}\n"}
      ],
      temperature = 0.8,
      n=1,
      max_tokens=30,
      frequency_penalty=1.19,
      presence_penalty=1.68
  )
  return response["choices"][0]["message"]["content"]

In [13]:
def get_ppt(industry: str, purpose: str, topic: str, tone: str, image = "AI", font_size = 16, slide_colour = RGBColor(80, 80, 80), font_colour = RGBColor(255, 255, 255), font_style = "Calibri"):
  ppt_info = generate_ppt(industry, purpose, topic, tone)
  # Create a new PowerPoint presentation
  presentation = Presentation()
  presentation.slide_width = Inches(13.33)
  presentation.slide_height = Inches(7.5)

  # Loop through the slides in the JSON data
  for key in ppt_info.keys():
    slide_data = ppt_info[key]
    if key == "title_slide":
      slide_layout = presentation.slide_layouts[0]
      slide = presentation.slides.add_slide(slide_layout)
      fill = slide.background.fill
      fill.solid()
      fill.fore_color.rgb = slide_colour

      title = slide.shapes.title
      title.text = slide_data["title"]
      title_text_frame = title.text_frame
      title_text_frame.paragraphs[0].runs[0].font.size = Pt(font_size + 4)
      title_text_frame.paragraphs[0].runs[0].font.color.rgb = font_colour
      title_text_frame.paragraphs[0].runs[0].font.bold = True
      title_text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

      subtitle_shape = slide.placeholders[1].text_frame
      subtitle_shape.text = slide_data["subtitle"]
      subtitle_shape.paragraphs[0].runs[0].font.size = Pt(font_size + 2)
      subtitle_shape.paragraphs[0].runs[0].font.color.rgb = font_colour
      subtitle_shape.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

    elif key == "outline_slide":
      slide_layout = presentation.slide_layouts[1]
      slide = presentation.slides.add_slide(slide_layout)
      fill = slide.background.fill
      fill.solid()
      fill.fore_color.rgb = slide_colour

      title = slide.shapes.title
      title.text = slide_data["title"]
      title_text_frame = title.text_frame
      title_text_frame.paragraphs[0].runs[0].font.size = Pt(font_size + 4)
      title_text_frame.paragraphs[0].runs[0].font.color.rgb = font_colour
      title_text_frame.paragraphs[0].runs[0].font.bold = True
      title_text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

      content = slide.placeholders[1].text_frame
      for bullet_point in slide_data["bullet_points"]:
          point = content.add_paragraph()
          point.text = bullet_point
          point.space_before = Inches(0.1)
          point.space_after = Inches(0.1)
          point.font.size = Pt(font_size)
          point.font.color.rgb = font_colour
          point.level = 0

      content.left = presentation.slide_width / 2
      content.top = presentation.slide_height / 2
      content.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

    else:
      # Add a new slide with title and content layout
      slide_layout = presentation.slide_layouts[1]
      slide = presentation.slides.add_slide(slide_layout)
      fill = slide.background.fill
      fill.solid()
      fill.fore_color.rgb = slide_colour

      # Set the slide title
      title = slide.shapes.title
      title.text = slide_data["title"]
      title_text_frame = title.text_frame
      title_text_frame.paragraphs[0].runs[0].font.size = Pt(font_size + 2)
      title_text_frame.paragraphs[0].runs[0].font.color.rgb = font_colour
      title_text_frame.paragraphs[0].runs[0].font.bold = True
      title_text_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

      # Add a text box
      text_box_width = Inches(4)
      text_box_height = Inches(2)
      text_box_left = Inches(2.5)
      text_box_top = Inches(1.5)
      text_Box = slide.shapes.add_textbox(text_box_left, text_box_top, text_box_width, text_box_height)
      tf = text_Box.text_frame
      tf.text = slide_data["description"]
      tf.paragraphs[0].runs[0].font.size = Pt(font_size)
      tf.paragraphs[0].runs[0].font.color.rgb = font_colour
      tf.paragraphs[0].alignment = PP_ALIGN.CENTER
      tf.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

      # Add bullet points
      content = slide.placeholders[1].text_frame
      for bullet_point in slide_data["bullet_points"]:
          point = content.add_paragraph()
          point.text = bullet_point
          point.space_before = Inches(0.1)
          point.space_after = Inches(0.1)
          point.font.size = Pt(font_size -  2)
          point.font.color.rgb = font_colour
          point.level = 0


          point_explainer = content.add_paragraph()
          if is_question(bullet_point):
            point_explainer.text = get_question_explainer(bullet_point, topic)
          else:
            point_explainer.text = get_point_explainer(bullet_point)
          point_explainer.font.size = Pt(font_size - 5)
          point_explainer.font.color.rgb = font_colour
          point_explainer.level = 1
    
      content.left = presentation.slide_width / 2
      content.top = presentation.slide_height / 2
      content.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE

      # Add image if available
      if slide_data["image"]:
          # Download the image (you need to have the 'requests' library installed)
          img = ""
          if image == "AI":
            img = get_ai_img_for_ppt(slide_data["image"])
          elif image == "Unsplash":
            response = requests.get(get_unsplash_img_for_ppt(slide_data["image"]))
            img = io.BytesIO(response.content)
          elif image == "Pixabay":
            response = requests.get(get_pixabay_img_for_ppt(slide_data["image"]))
            img = io.BytesIO(response.content)
          elif image == "Pexels":
            response = requests.get(get_pexels_img_for_ppt(slide_data["image"]))
            img = io.BytesIO(response.content)
          elif image == "Flickr":
            response = requests.get(get_flickr_img_for_ppt(slide_data["image"]))
            img = io.BytesIO(response.content)

          # Add the image to the slide
          left = Inches(9.5)
          top = presentation.slide_height / 4
          height = presentation.slide_height / 2
          slide.shapes.add_picture(img, left, top, height=height)

  # Save the presentation to a file
  return presentation

In [ ]:
ppt = get_ppt(industry, purpose, topic, tone)
ppt.save("ppt1.pptx")

Ignoring args : ('This must be treated as a question.',)
